In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO # String을 파일처럼 다룰 수 있는 도구

In [20]:
csv_data = """A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
9.0,10.0,11.0,
"""
sio = StringIO(csv_data)
df = pd.read_csv(sio)

In [21]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,9.0,10.0,11.0,NaN


In [4]:
# missing value 확인

# df.info()
df.isna()
df.isna().sum() # True : 1, False : 0
df.isnull() # isna()와 동일
df.isnull().sum()
df.notna()
df.notnull()

,A,B,C,D
0,True,True,True,True
1,True,True,False,True
2,True,True,True,False


In [5]:
# missing value 처리 1. 삭제 ( 행 또는 열 )

# df.dropna() # NaN이 포함된 모든 행 제거 : axis=0
# df.dropna(axis=1) # NaN이 포함된 모든 열 제거
# df.dropna(how="all") # 모든 데이터가 NaN인 행 제거 (any or all)
# df.dropna(thresh=4) # NaN이 아닌 데이터가 4개 미만인 행 제거
# df.dropna(subset=['C']) # C 컬럼에 NaN이 포함된 행 제거

In [6]:
# missing value 처리 2. 대체
df
c_mean = df["C"].mean()
print(c_mean)
df["C"].fillna(c_mean)
d_median = df['D'].median()
print(d_median)
df['D'].fillna(d_median)

7.0
6.0


0    4.0
1    8.0
2    6.0
Name: D, dtype: float64

In [7]:
# missing value 처리 2. 대체
from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy="mean")
# imputer = SimpleImputer(strategy="median")
# imputer = SimpleImputer(strategy="most_frequent")
imputer = SimpleImputer(strategy="constant", fill_value=100)
imputer.fit(df[['C']]) # strategy 에따라 변환할 값 계산
imputer.transform(df[['C']])

array([[  3.],
       [100.],
       [ 11.]])

In [26]:
df2 = pd.DataFrame([
    ['green', 'M', 10.1, 'class1', 1],
    ['red', 'L', 13.5, 'class2', 2],
    ['blue', 'XL', 15.3, 'class1', 1],
])

df2.columns = ['color', 'size', 'price', 'classlabel', 'cls']
df2

,color,size,price,classlabel,cls
0,green,M,10.1,class1,1
1,red,L,13.5,class2,2
2,blue,XL,15.3,class1,1


In [9]:
np.unique(df2['classlabel'])

array(['class1', 'class2'], dtype=object)

In [10]:
# 범주형 데이터 -> 수치형 (이산형) 데이터 변환 ( 직접 변환 논리 구현 )
class_mapping = { 'class1' : 0, 'class2' : 1, 'class3' : 2 }
# class_mapping = { label : idx for idx, label in enumerate(np.unique(df2['classlabel'])) }

df2['classlabel'] = df2['classlabel'].map(class_mapping)
df2

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [11]:
# 수치형(이산형) 데이터 -> 범주형 데이터로 역변환 ( 직접 변환 논리 구현 )
inv_class_mapping = { v : k for k, v in class_mapping.items() }
# print(inv_class_mapping)
df2['classlabel'] = df2['classlabel'].map(inv_class_mapping)
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [12]:
# 범주형 데이터 -> 수치형 (이산형) 데이터 변환 ( 도구 활용 )
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df2["classlabel"]) # 변환 규칙 학습
transformed_data = le.transform(df2["classlabel"]) # 학습한 변환 규칙에 따라 변환
print( transformed_data )
df2["classlabel"] = transformed_data
df2

[0 1 0]


,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [13]:
# 수치형(이산형) 데이터 -> 범주형 데이터로 역변환 ( 도구 활용 )
print( le.inverse_transform(transformed_data) )
df2['classlabel'] = le.inverse_transform(df2['classlabel'])
df2

['class1' 'class2' 'class1']


,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [14]:
# 명목형 데이터에 대해 LabelEncoder를 사용할 때의 문제 상황
le2 = LabelEncoder()
le2.fit(df2['color'])
transformed = le2.transform(df2['color'])
transformed

array([1, 2, 0])

In [15]:
df2['color']

0    green
1      red
2     blue
Name: color, dtype: object

In [25]:
# 명목형 데이터 -> 수치형 데이터로 변환 ( OneHotEncoding )
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe.fit(df2[['color']])
transformed_data = ohe.transform(df2[['color']])
# transformed_data
# print(transformed_data) # sparse matrix
transformed_data.toarray() # dense matrix


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [27]:
# 명목형 데이터 -> 수치형 데이터로 변환 ( OneHotEncoding )
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe.fit(df2[['cls']])
transformed_data = ohe.transform(df2[['cls']])
# transformed_data
# print(transformed_data) # sparse matrix
transformed_data.toarray() # dense matrix

array([[1., 0.],
       [0., 1.],
       [1., 0.]])

In [30]:
# 명목형 데이터 -> 수치형 데이터로 변환 ( OneHotEncoding )
pd.get_dummies(df2['color'], drop_first=True)

,green,red
0,True,False
1,False,True
2,False,False


In [34]:
# df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning/databases/wine/wine.data', header=None)
df_wine = pd.read_csv("data-files/wine.data")
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [36]:
df_wine['Class label'].value_counts()

Class label
2    71
1    58
3    48
Name: count, dtype: int64

In [41]:
# 연속형 데이터의 값 크기 조정 1 : 속성의 값의 범위를 지정해서 변경 (보통 0 ~ 1)
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
mms.fit(df_wine[1:])
scaled_data = mms.transform(df_wine[1:])

scaled_data.min(axis=0), scaled_data.max(axis=0)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [43]:
# 연속형 데이터의 값 크기 조정 2 : 속성의 값의 분포를 표준 정규분포로 변경
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(df_wine[1:])
scaled_data = ss.transform(df_wine[1:])

scaled_data.mean(axis=0), scaled_data.std(axis=0), scaled_data.min(axis=0), scaled_data.max(axis=0) 

(array([ 0.00000000e+00,  3.22973971e-16, -1.21115239e-16,  8.27620800e-16,
        -6.05576195e-16,  4.64275083e-16,  2.82602224e-16,  2.42230478e-16,
        -8.07434927e-17, -2.62416351e-16, -2.22044605e-16, -4.03717464e-16,
        -4.03717464e-16, -4.03717464e-17]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([-1.23117379, -2.42686016, -1.43321246, -3.66945174, -2.73607884,
        -2.08729084, -2.09341224, -1.68413805, -1.8719762 , -2.06399541,
        -1.62599856, -2.080341  , -1.89282676, -1.48186387]),
 array([1.36387516, 2.27228308, 3.09064693, 3.14172617, 3.18524181,
        4.40347414, 2.54005527, 3.06962723, 2.38803698, 3.48695335,
        3.41727666, 3.2902016 , 1.99311325, 2.98254414]))

In [50]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_wine, test_size=0.2, random_state=42)

train_set.shape, test_set.shape

((141, 14), (36, 14))

In [51]:
from sklearn.model_selection import train_test_split

X = df_wine.drop("Class label", axis=1)
y = df_wine['Class label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((141, 13), (141,), (36, 13), (36,))